# ESAT 2022 - FeO$_\text{s}$ demonstration

**Try it yourself! https://github.com/feos-org/demo/esat2022**

## General Workflow

In [ ]:
#@title Download FeO$_\text{s}$ and parameter files
!pip install feos
!wget https://raw.githubusercontent.com/feos-org/feos/main/parameters/pcsaft/loetgeringlin2018.json
!cat /proc/cpuinfo

In [ ]:
#@title Import Python packages

from feos.si import *     # SI units and constants
from feos.eos import *    # equation of state objects
from feos.pcsaft import * # parameter objects

import numpy as np              # for arrays
import matplotlib.pyplot as plt # plotting utilities
import seaborn as sns
sns.set_style("ticks")
sns.set_context("poster")
sns.set_palette("Dark2")

In [ ]:
#@title 1. Define Model
parameters = PcSaftParameters.from_json(
    ["hexane", "toluene"], 
    "loetgeringlin2018.json"
)
pcsaft = EquationOfState.pcsaft(parameters)

In [ ]:
#@title 2. Define Thermodynamic Conditions
temperature = 315.0 * KELVIN
pressure = 2.0 * BAR
x = np.array([0.5 , 0.5])
state = State(
    pcsaft, 
    temperature=temperature,
    pressure=pressure,
    molefracs=x,
)
state

In [ ]:
#@title 3. Compute Properties
state.volume / (MILLI*METER)**3

## Critical Points & Phase Equilibria

In [ ]:
#@title State at Critical Conditions
state_cp = State.critical_point(
    pcsaft, 
    moles=np.array([1.0, 1.0]) * MOL
)
state_cp

In [ ]:
state_cp.pressure()

In [ ]:
#@title Compute phase diagram
phase_diagram = PhaseDiagram.binary_vle(
    pcsaft,
    500 * KELVIN,
    npoints=251
)

In [ ]:
#@title Plot phase diagram
KG_M3 = KILOGRAM / METER**3
plt.figure(figsize=(12, 7))
plt.plot(
    phase_diagram.vapor.molefracs[:, 0],
    phase_diagram.vapor.pressure / BAR
)
plt.plot(
    phase_diagram.liquid.molefracs[:, 0],
    phase_diagram.liquid.pressure / BAR
)
plt.ylabel(r"$p$ / bar")
plt.xlabel(r"$x_1$")
plt.xlim(0, 1)
plt.ylim(12.0, 28.0);

In [ ]:
vle = PhaseEquilibrium.tp_flash(pcsaft, 500*KELVIN, 16*BAR, np.array([0.3, 0.7]) * MOL)
vle

In [ ]:
bubble = PhaseEquilibrium.bubble_point(
    pcsaft, 
    500*KELVIN, 
    vle.liquid.molefracs,
)
bubble

In [ ]:
bubble.liquid.chemical_potential() - bubble.vapor.chemical_potential()

In [ ]:
#@title Entropy scaling
#todo

## Classical Density Functional Theory (DFT)

In [ ]:
from feos.dft import *    # classical density function theory objects
func = HelmholtzEnergyFunctional.pcsaft(parameters)

In [ ]:
#@title Surface tension
#@markdown 1. compute VLE
#@markdown 2. set initial density profile
#@markdown 3. solve for equilibrium profile

vle = PhaseEquilibrium.tp_flash(
    func,
    500 * KELVIN,
    16 * BAR,
    np.array([0.3, 0.7]) * MOL
)
interface = PlanarInterface.from_tanh(
    vle, 1024, 100 * ANGSTROM, 500 * KELVIN
).solve()

In [ ]:
interface.surface_tension

In [ ]:
#@title Density profiles
KMOL_M3 = KILO * MOL / METER**3
plt.figure(figsize=(12, 7))
plt.plot(interface.z / ANGSTROM, (interface.density / KMOL_M3)[0], 
         label="{}".format(parameters.pure_records[0].identifier.name))
plt.plot(interface.z / ANGSTROM, (interface.density / KMOL_M3)[1],
         label="{}".format(parameters.pure_records[1].identifier.name))
plt.xlabel(r"$z$ / A");
plt.ylabel(r"$\rho$ / kmol / m$^3$")
plt.ylim(0, 5);
plt.xlim(0, 100)
plt.legend(frameon=False);

In [ ]:
#@title Adsorption in pore
potential = ExternalPotential.LJ93(3.0, 100.0, 0.08)

pore = Pore1D(
    geometry=Geometry.Cartesian, 
    pore_size=40 * ANGSTROM, 
    potential=potential
)

isotherm = Adsorption1D.equilibrium_isotherm(
    func,
    temperature=500 * KELVIN,
    pressure=SIArray1.linspace(6*BAR, 22*BAR, 5),
    pore=pore,
    molefracs=np.array([0.5, 0.5])
)

In [ ]:
#@title Adsorpted amount per surface
plt.figure(figsize=(12, 7))
plt.title(r"$T$ = 500 K")
plt.plot(
    isotherm.pressure/BAR, 
    isotherm.total_adsorption/(MICRO*MOL/METER**2)
)
plt.xlim(6, 22)
plt.ylim(1, 12)
plt.xlabel(r'$p$ / bar')
plt.ylabel(r'$N$ /  $\mu$mol / m$^2$');

In [ ]:
#@title Density profiles
KMOL_M3 = KILO * MOL / METER**3
i = 2 #@param {type:"slider", min:0, max:6, step:1}
z = isotherm.profiles[0].z / ANGSTROM
p = isotherm.pressure / BAR
total_adsorption = isotherm.total_adsorption / (MICRO * MOL / METER**2)

# figure
fig, ax = plt.subplots(
    1, 2, figsize=(15, 5), 
    gridspec_kw={'wspace': 0.25}
)

# isotherm
ax[0].plot(p, total_adsorption)
ax[0].plot(p[i], total_adsorption[i], marker="s", clip_on=False)
ax[0].set_xlim(6, 22)
ax[0].set_xticks(range(6, 23, 4))
ax[0].set_ylim(1, 12)
ax[0].set_xlabel(r'$p$ / bar')
ax[0].set_ylabel(r'$N$ /  $\mu$mol / m$^2$');

# density profile
ax[1].plot(z, (isotherm.profiles[i].density / KMOL_M3)[0], 
         label="{}".format(parameters.pure_records[0].identifier.name))
ax[1].plot(z, (isotherm.profiles[i].density / KMOL_M3)[1], 
         label="{}".format(parameters.pure_records[1].identifier.name))
ax[1].set_xlim(0, 20)
ax[1].set_ylim(0, 12)
ax[1].set_xlabel(r"$z$ / A")
ax[1].set_ylabel(r"$\rho$ / kmol / m$^3$")
ax[1].legend(frameon=False, loc="upper left");